#  Incident Root Cause Analysis 

Incident Reports in ITOps usually states the symptoms. Identifying the root cause of the symptom quickly is a key determinant to reducing resolution times and improving user satisfaction.

In [2]:
#Install all related packages. If you find additional packages missing, please follow the same technique.
#If you are not using anaconda, then use pip to install the same packages

import sys
!conda install --yes --prefix {sys.prefix} pandas tensorflow scikit-learn

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: | ^C
failed

CondaError: KeyboardInterrupt



# 02.04. Preprocessing Incident Data

### Loading the Dataset

In [32]:
import pandas as pd
import os
import tensorflow as tf

#Load the data file into a Pandas Dataframe
symptom_data = pd.read_csv("root_cause_analysis.csv")

#Explore the data loaded
print(symptom_data.dtypes)
symptom_data.head()

ID              int64
CPU_LOAD        int64
MEMORY_LOAD     int64
DELAY           int64
ERROR_1000      int64
ERROR_1001      int64
ERROR_1002      int64
ERROR_1003      int64
ROOT_CAUSE     object
dtype: object


,ID,CPU_LOAD,MEMORY_LOAD,DELAY,ERROR_1000,ERROR_1001,ERROR_1002,ERROR_1003,ROOT_CAUSE
0,1,0,0,0,0,1,0,1,MEMORY
1,2,0,0,0,0,0,0,1,MEMORY
2,3,0,1,1,0,0,1,1,MEMORY
3,4,0,1,0,1,1,0,1,MEMORY
4,5,1,1,0,1,0,1,0,NETWORK_DELAY


In [33]:
symptom_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           1000 non-null   int64 
 1   CPU_LOAD     1000 non-null   int64 
 2   MEMORY_LOAD  1000 non-null   int64 
 3   DELAY        1000 non-null   int64 
 4   ERROR_1000   1000 non-null   int64 
 5   ERROR_1001   1000 non-null   int64 
 6   ERROR_1002   1000 non-null   int64 
 7   ERROR_1003   1000 non-null   int64 
 8   ROOT_CAUSE   1000 non-null   object
dtypes: int64(8), object(1)
memory usage: 70.4+ KB


### Convert  data

Input data needs to be converted to formats that can be consumed by ML algorithms

In [34]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
symptom_data['ROOT_CAUSE'] = label_encoder.fit_transform(
                                symptom_data['ROOT_CAUSE'])

#Convert Pandas DataFrame to a numpy vector
np_symptom = symptom_data.to_numpy().astype(float)

#Extract the feature variables (X)
X_train = np_symptom[:,1:8]

#Extract the target variable (Y), convert to one-hot-encoding
Y_train=np_symptom[:,8]
Y_train = tf.keras.utils.to_categorical(Y_train,3)

print("Shape of feature variables :", X_train.shape)
print("Shape of target variable :",Y_train.shape)

Shape of feature variables : (1000, 7)
Shape of target variable : (1000, 3)


## 02.05. Building the Model with Keras

In [35]:
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.regularizers import l2

#Setup Training Parameters
EPOCHS=20
BATCH_SIZE=100
VERBOSE=1
OUTPUT_CLASSES=len(label_encoder.classes_)
N_HIDDEN=128
VALIDATION_SPLIT=0.2

#Create a Keras sequential model
model = tf.keras.models.Sequential()
#Add a Dense Layer
model.add(keras.layers.Dense(N_HIDDEN,
                             input_shape=(7,),
                              name='Dense-Layer-1',
                              activation='relu'))

#Add a second dense layer
model.add(keras.layers.Dense(N_HIDDEN,
                              name='Dense-Layer-2',
                              activation='relu'))

#Add a softmax layer for categorial prediction
model.add(keras.layers.Dense(OUTPUT_CLASSES,
                             name='Final',
                             activation='softmax'))

#Compile the model, using Adam optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#Build the model
model.fit(X_train,
          Y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          verbose=VERBOSE,
          validation_split=VALIDATION_SPLIT)

model.summary()

Epoch 1/20
8/8 [==============================] - 0s 19ms/step - loss: 1.0180 - accuracy: 0.5675 - val_loss: 0.9543 - val_accuracy: 0.7100
Epoch 2/20
8/8 [==============================] - 0s 4ms/step - loss: 0.8504 - accuracy: 0.7900 - val_loss: 0.8063 - val_accuracy: 0.7400
Epoch 3/20
8/8 [==============================] - 0s 4ms/step - loss: 0.6967 - accuracy: 0.8125 - val_loss: 0.6824 - val_accuracy: 0.7500
Epoch 4/20
8/8 [==============================] - 0s 4ms/step - loss: 0.5714 - accuracy: 0.8138 - val_loss: 0.5940 - val_accuracy: 0.7600
Epoch 5/20
8/8 [==============================] - 0s 4ms/step - loss: 0.4866 - accuracy: 0.8300 - val_loss: 0.5469 - val_accuracy: 0.7900
Epoch 6/20
8/8 [==============================] - 0s 4ms/step - loss: 0.4515 - accuracy: 0.8350 - val_loss: 0.5353 - val_accuracy: 0.8000
Epoch 7/20
8/8 [==============================] - 0s 4ms/step - loss: 0.4350 - accuracy: 0.8413 - val_loss: 0.5342 - val_accuracy: 0.8000
Epoch 8/20
8/8 [=================

## 02.06. Predicting Root Causes

In [41]:
#Pass individual flags to Predict the root cause
CPU_LOAD=1
MEMORY_LOAD=0
DELAY=0
ERROR_1000=0
ERROR_1001=1
ERROR_1002=1
ERROR_1003=0

prediction=model.predict(
    [[CPU_LOAD,MEMORY_LOAD,DELAY,
      ERROR_1000,ERROR_1001,ERROR_1002,ERROR_1003]])

predicted = np.argmax(prediction,axis=1)

print(label_encoder.inverse_transform(predicted))

1/1 [==============================] - 0s 27ms/step
['DATABASE_ISSUE']


In [44]:
#Predicting as a Batch
print(label_encoder.inverse_transform(
        np.argmax(model.predict([[1,0,0,0,1,1,0],
                                [0,1,1,1,0,0,0],
                                [1,1,0,1,1,0,1],
                                [0,0,0,0,0,1,0],
                                [1,0,1,0,1,1,1]]),axis=1)))

1/1 [==============================] - 0s 28ms/step
['DATABASE_ISSUE' 'NETWORK_DELAY' 'MEMORY' 'DATABASE_ISSUE'
 'DATABASE_ISSUE']


In [25]:
import sklearn

print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 1.1.2.


In [22]:
import pycaret
from pycaret.classification import *